# Model Building
Where I build the CNN model

In [12]:
import sys
import os

from dotenv import load_dotenv

import torch
import torch.nn as nn
import torch.nn.functional as F

# add parent directory to path so I can import from data)
sys.path.append(os.getenv('HOME_DIR'))

from data.import_data import load_dataset, get_dataloader

load_dotenv()

True

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loss_func = nn.BCELoss()

In [19]:
class Model(nn.Module):
    '''
    A CNN with 1 convolutional layer and 1 linear layer.
    '''
    def __init__(self, num_channels):
        super().__init__()
                
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, 
                      out_channels=num_channels, 
                      kernel_size=(3, 3),
                      stride=1,),
            nn.BatchNorm2d(num_channels),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2), stride=2, padding=0),
            nn.Dropout(0.2)
        )
        self.layer2 = nn.Linear(num_channels * 63 * 39, 2)

        
    def forward(self, x):
        x = self.layer1(x)
        x = torch.flatten(x, 1)
        x = F.softmax(self.layer2(x), dim=1)
        return x

In [20]:
model = Model(num_channels=16).to(device)

In [21]:
# view model parameters
for name, param in model.named_parameters():
    print(f"Name: {name}")
    print(f"Size: {param.size()}")

Name: layer1.0.weight
Size: torch.Size([16, 1, 3, 3])
Name: layer1.0.bias
Size: torch.Size([16])
Name: layer1.1.weight
Size: torch.Size([16])
Name: layer1.1.bias
Size: torch.Size([16])
Name: layer2.weight
Size: torch.Size([2, 39312])
Name: layer2.bias
Size: torch.Size([2])


In [22]:
train_ds = load_dataset(os.path.join(os.getenv('HOME_DIR'),'data/meta/train.csv'), files_to_load=100)
train_dl = get_dataloader(train_ds, batch_size=32)

In [23]:
for batch, labels in train_dl:


    batch = batch.to(device)
    labels = labels.to(device)
    output = model(batch)
    loss = loss_func(output, labels)
    #print(f"Loss: {loss}")
    print(output)
    print(labels)

    break

tensor([[0.8131, 0.1869],
        [0.5021, 0.4979],
        [0.4771, 0.5229],
        [0.5163, 0.4837],
        [0.4988, 0.5012],
        [0.5371, 0.4629],
        [0.0423, 0.9577],
        [0.6550, 0.3450],
        [0.4921, 0.5079],
        [0.3390, 0.6610],
        [0.5045, 0.4955],
        [0.5314, 0.4686],
        [0.4792, 0.5208],
        [0.5360, 0.4640],
        [0.9894, 0.0106],
        [0.5128, 0.4872],
        [0.2789, 0.7211],
        [0.4537, 0.5463],
        [0.5783, 0.4217],
        [0.4519, 0.5481],
        [0.4974, 0.5026],
        [0.7242, 0.2758],
        [0.5069, 0.4931],
        [0.5025, 0.4975],
        [0.5098, 0.4902],
        [0.4919, 0.5081],
        [0.4932, 0.5068],
        [0.4976, 0.5024],
        [0.4860, 0.5140],
        [0.5499, 0.4501],
        [0.5062, 0.4938],
        [0.4925, 0.5075]], grad_fn=<SoftmaxBackward0>)
tensor([[0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 

In [18]:
model

Model(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Linear(in_features=39312, out_features=2, bias=True)
)